In [181]:
import pandas as pd
import numpy as np
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# **Analysis Pipleine - Group 23 - David Olson**
---
- (note: This pipeline will continue to be updated as I work on ALL 4 milestones.)
---
## **1. Load Data**
### Steps taken:
- Loaded data, there is a difference in date. SPY has values from 1993-2020, and TSLA has values from 2010-2020.
- I will remove 1993-2010 from spy so the dates tracked are the same as TSLA's
- There are initially 7 columns in each dataframe, I aim to get that to 7 columns total as a merged dataframe.
- To do this, I will have to remove some columns that I think are least necessary.
- add link for:
    - milestone1.ipynb (these are the initial 2 dataframes.)
---
## **2. Clean Data** (Note: Steps 2,3, and 4 link together and there may be some cleaning/processing/wrangling steps in all 3 steps)
### Steps taken:
- I created a function with method-chaining called "load_and_clean_tsla_spy" that loads the dataframes, removes the "High", "Low", and "Adj. Close" columns, and renames the "Open", "Close", and "Volume" columns to "open_spy" or "open_tsla", "close_spy" or "close_tsla", and "vol_spy" or "vol_tsla" from both dataframes. It also converts the date column to datetime.
---
```
def load_and_clean_tsla_spy(spy,tsla):
    tsladf = (pd.read_csv(tsla, parse_dates=["Date"])
              .drop(columns=["High","Low","Adj Close"])
              .rename(columns={"Open":"open_tsla","Close":"close_tsla","Volume":"vol_tsla"})
              .reset_index(drop=True))
    spydf = (pd.read_csv(spy, parse_dates=["Date"])
             .drop(columns=["High","Low","Adj Close"])
             .rename(columns={"Open":"open_spy","Close":"close_spy","Volume":"vol_spy"})
             .reset_index(drop=True))
    return (spydf,tsladf)
```
- add link for:
    - analysis/scripts
    - FunctionWithMC.ipynb
---
## **3. Process Data**
### Steps taken:
- I created another function with method-chaining called "remove_dates" that allows me to remove the dates that are out of my specified range in the SPY dataframe. This function is to be run after "load_and_clean_tsla_spy".
---
```
x=load_and_clean_tsla_spy(spy,tsla)
spydf=x[0]

removeolder=pd.to_datetime("2010-06-28")
removenewer=pd.to_datetime("2020-02-04")
def remove_dates(spydf):
    spyclean = (pd.DataFrame(spydf)
               .loc[(spydf.Date > removeolder)]
                .loc[(spydf.Date < removenewer)]
                .reset_index(drop=True))
    return spyclean
```
- add link for:
    - analysis/scripts
    - FunctionWithMC.ipynb
---
## **4. Wrangle Data**
### Steps taken:
- I created a third function with method-chaining called "merge_n_round" that takes cleaned/processed dataframes, merges them together, and rounds the numbers to 2 decimal places for ease of reading.
- I also added 2 columns: daygain_tsla and daygain_spy 
---
```
y=remove_dates(spydf)
spydf=y
tsladf=x[1]
tslaspymerged=(spydf,tsladf)

def merge_n_round(tslaspymerged):
    
    tslaspymerged = (pd.DataFrame(pd.merge(spydf,tsladf, how = 'inner'))
                     .round(decimals=2))
    return tslaspymerged
```
- add link for: 
    - analysis/scripts
    - FunctionWithMC.ipynb
---
## **5. Exploratory Data analysis**
### Steps taken:
- For this task, I imported my 3 project functions and then used the cleaned dataframe to conduct exploratory data analysis
---
### Plot 1:
- I created a plot out of the dataframe that shows the daily volume of SPY & TSLA from 2010-2020.
---

```

edadf=finalmergeddf[['Date','vol_tsla','vol_spy']]
df = pd.DataFrame(data=edadf)

ax = df.plot(x="Date", y="vol_tsla", legend=True, color="green", figsize=(18,10))

ax2 = ax.twinx()
df.plot(x="Date", y="vol_spy", ax=ax2, legend=True, color="r",figsize=(18,10))
ax.figure.legend()
plt.show()

```
---
- I was interested in seeing how much of a change there was in volume from 2010-2020 for both.
- I then used the data to print out the mean volume for the first 365 days and last 365 days of the dataframe.
---
```
tslavolfirst365="TSLA's mean volume for the first 365 days traded on the dataset is:",np.mean(df.vol_tsla.head(365))
tslavollast365="TSLA's mean volume for the last 365 days traded on the dataset is:",np.mean(df.vol_tsla.tail(365))
spyvolfirst365="SPY's mean volume for the first 365 days traded on the dataset is:",np.mean(df.vol_spy.head(365))
spyvollast365="SPY's mean volume for the last 365 days traded on the dataset is:",np.mean(df.vol_spy.tail(365))

print(tslavolfirst365)
print(tslavollast365)
print(spyvolfirst365)
print(spyvollast365)
```
---
- The output is as follows:

```

("TSLA's mean volume for the first 365 days traded on the dataset is:", 1406631.7808219178)
("TSLA's mean volume for the last 365 days traded on the dataset is:", 9843472.87671233)
("SPY's mean volume for the first 365 days traded on the dataset is:", 209259411.50684932)
("SPY's mean volume for the last 365 days traded on the dataset is:", 79445112.05479452)

```
---
### Plot 2:
- I created another plot because out of personal interest, I simply wanted to see the TSLA/SPY close price together on one chart.
---


```

dfx=finalmergeddf

df = pd.DataFrame(data=dfx)

ax = df.plot(x="Date", y="close_tsla",color="green", figsize=(8,8))

ax2 = ax.twiny()
df.plot(x='Date', y="close_spy", ax=ax2, color="r",figsize=(8,8))
plt.title('Closing prices of tsla and spy from 2010-2020')
plt.show()

```
---
### Plot 3:
- I created a plot called daily gain that shows the daily gain/loss for SPY/TSLA (using closeprice-openprice).
- I wanted to see how much more volatile the TSLA daygain was vs. SPY's daygain.
---
```

dfx['daygain_tsla']=dfx['close_tsla']-dfx['open_tsla']
dfx['daygain_spy']=dfx['close_spy']-dfx['open_spy']
df = pd.DataFrame(data=dfx)

ax = df.plot(x="Date", y="daygain_tsla",color="orange", figsize=(8,8),kind='line')

ax2 = ax.twiny()
df.plot(x='Date', y="daygain_spy", ax=ax2, color="blue",figsize=(8,8),kind='line')
plt.title('Daily gain (close-open) for TSLA & SPY')
plt.ylabel('daily gain/loss in USD')

plt.show()

```
---

 


- add links of:
    -EDA(Task3).ipynb
        -Plot 1
        -Plot 2
        -Plot 3
---
## **6. Data Analysis**
### Steps taken:
- For this task, I used my imported project functions to clean the dataframes and merge them. I then created a few visualizations to help answer my research questions in relation to volume and daygain of TSLA and SPY.
- I also added a bunch more plots after receiving feedback from milestone 2.
- Consists of 7 figures and various volume/gain/analyses
- The variables that I plan on analyzing include: Date, Open, Close, and Volume. 
- Potential questions I plan on answering are: "Does volume effect/correlate with the close price for tsla or spy?" 
- "Analysis of volume increase/decrease SPY/TSLA"
- I am also going to analyze daygain/volume/date/close price and if there are any correlations between the variables. 

#### FIGURE 1 Daygain and volume: 
- plot 1: All days of data set (2010-2020)
- plot 2: 1st 365 days of data set (2010-2011)
- plot 3: last 365 days of data set (2019-2020)
#### FIGURE 2 Volume by date for TSLA and SPY:
- plot 1: Volume by date for TSLA, from entire dataset (2010-2020)
- plot 2: Volume by date for SPY, from entire dataset (2010-2020)
- plot 3: Volume by date for TSLA, first 365 days from dataset (~2010-2011)
- plot 4: Volume by date for SPY, first 365 days from dataset (~2010-2011)
- plot 5: Volume by date for TSLA, last 365 days from dataset (~2019-2020)
- plot 6: Volume by date for SPY, last 365 days from dataset (~2019-2020)
#### FIGURE 3 More volume analysis between TSLA and SPY (2 y axis):
- plot 1: TSLA vs. SPY vol analysis, entire data set
- plot 2: TSLA vs. SPY vol analysis, first 365 days of dataset.
- plot 3: TSLA vs. Spy vol analysis, last 365 days of dataset.
#### FIGURE 4 TSLA/SPY Volume and Close price by Date:
- plot 1: TSLA/SPY Volume and Close price by Date from entire dataset
- plot 2: TSLA/SPY Volume and Close price by Date from first 365 days of dataset
- plot 3: TSLA/SPY Volume and Close price by Date from last 365 days of dataset
#### FIGURE 5 Daygain for TSLA/SPY by date.
- plot 1: Daygain for TSLA/SPY by date - entire dataset.
- plot 2: Daygain for TSLA/SPY by date - 1st 365 days.
- plot 3: Daygain for TSLA/SPY by date - last 365 days.
#### FIGURE 6 More volume analysis between TSLA and SPY (1 Y-Axis).
- plot 1: TSLA vs. SPY vol analysis, entire data set
- plot 2: TSLA vs. SPY vol analysis, first 365 days of dataset.
- plot 3: TSLA vs. Spy vol analysis, last 365 days of dataset.
#### FIGURE 7 Close price TSLA/SPY by Date:
- plot 1: Close price TSLA/SPY by Date - entire dataset
- plot 2: Close price TSLA/SPY by Date - 1st 365 days
- plot 3: Close price TSLA/SPY by Date - last 365 days

- add links: TASK4.ipynb 
---
## **7. Export reports/data analyses and visualizations (not required for this Task)**
### Steps taken:
---
## **8. Address feedback from T.A**
---
### Feedback from T.A:
- What I did in regards to the feedback will be shown after below after '------'

```
In General
[1] It is highly recommended that you justify your project. Please note that the Jupyter notebook supports markdown, and you can use this feature to  make your project clearer.
------ Updated analysis pipeline and various readme.md's to act more as a guide and make my project more clear. Also justified the questions in the main readme.md  
Task 1
[1] good work
------ Added explanation of Task 1&2 to pipeline.
Task 2
[1] Nice work
------ Added explanation of Task 1&2 to pipeline.
Task 3
[1 ] You need to use comments to justify your work.
------ I added my reasoning for why I chose to use these charts in Step 5 of the AnalysisPipeline.
[2] While you have created very good charts, it is hard to understand why you have generated them.
------ I added my reasoning for why I chose to use these charts in Step 5 of the AnalysisPipeline.
Task 4
[1] You need more charts to support your research question.
------ added way more charts
[2] The charts that you have used seems not proper for your research question
------ tweaked research question a little bit and added different charts.
[3] Appart from chart you need some reports, which need to be created using python
------ WIP

```

In [240]:
#Task 1 - Step 1 & 2 here

#Used: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html
#Used: https://stackoverflow.com/questions/58867452/drop-rows-from-pandas-dataframe-according-to-date
spy = "~/project-group23-project/data/raw/spy.csv"
tsla = "~/project-group23-project/data/raw/TSLA.csv"

spydf=pd.read_csv(spy)
tsladf=pd.read_csv(tsla)

#print(tsladf.describe().T)
#print(spydf.describe().T)
#print(len(spydf))
#print(len(tsladf))
#spydf.to_datetime
#spydf.dtypes

spydf['Date']=pd.to_datetime(spydf["Date"])


spydf = spydf[spydf.Date>pd.to_datetime('2010-06-28')]
spydf = spydf[spydf.Date<pd.to_datetime('2020-02-04')]
spydf = spydf.reset_index()
spydf = spydf.drop(columns = 'index')
spydf

# Here I converted spydf's 'Date' column into a datetime object
# I then made the range of dates from spydf match the tsladf.
# 2010-06-29 is the first date on BOTH dataframes
# 2020-02-03 is the last date on BOTH dataframes
# Both dataframes are now 2416 rows x 7 columns



,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,106.019997,107.510002,103.550003,104.209999,84.502396,373649500
1,2010-06-30,103.919998,104.879997,102.879997,103.220001,83.699646,284101700
2,2010-07-01,103.150002,103.489998,101.129997,102.760002,83.326614,382924800
3,2010-07-02,103.110001,103.419998,101.620003,102.199997,82.872505,233385200
4,2010-07-06,103.639999,104.370003,101.879997,102.870003,83.415825,256935300
...,...,...,...,...,...,...,...
2411,2020-01-28,325.059998,327.850006,323.600006,326.890015,320.897308,63834000
2412,2020-01-29,328.380005,328.630005,326.399994,326.619995,320.632233,53888900
2413,2020-01-30,324.359985,327.910004,323.540009,327.679993,321.672821,75491800
2414,2020-01-31,327.000000,327.170013,320.730011,321.730011,315.831909,113845600


In [187]:
# Task 1 - Step 3 HERE:
# I plan on removing the "High", "Low", "Adj Close" columns from both dataframes as of right now
#spydf = spydf.drop(columns=["High","Low","Adj Close"])
#tsladf = tsladf.drop(columns=["High","Low","Adj Close"])
# They are now both 2416 rows x 4 columns. 
# I am going to also convert TSLA to datetime
#tsladf['Date']=pd.to_datetime(tsladf["Date"])


In [216]:
# Task 1 - Step 4 here
# I plan on merging the two dataframes into one.
# However, after a few attempts, I realized the column names are equivalent on both dataframes, so I will have to rename the columns.

# USED: https://stackoverflow.com/questions/11346283/renaming-column-names-in-pandas
# USED: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html
# USED: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html
# USED: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.round.html
#spydf=spydf.rename(columns = {"Open":"open_spy","Close":"close_spy","Volume":"vol_spy"})
#tsladf=tsladf.rename(columns = {"Open":"open_tsla", "Close":"close_tsla","Volume":"vol_tsla"})
# I renamed a few columns to prepare for merging to a new dataframe.

tslaspydf = pd.DataFrame(pd.merge(tsladf,spydf,how='inner'))
tslaspydf = tslaspydf.round(decimals = 2)
tslaspydf
# the two dataframes are now merged into a dataframe called "tslaspydf"
# I also rounded the decimals to 2 places because it looks a lot nicer than 8.


,Date,open_tsla,close_tsla,vol_tsla,open_spy,close_spy,vol_spy
0,2010-06-29,19.00,23.89,18766300,106.02,104.21,373649500
1,2010-06-30,25.79,23.83,17187100,103.92,103.22,284101700
2,2010-07-01,25.00,21.96,8218800,103.15,102.76,382924800
3,2010-07-02,23.00,19.20,5139800,103.11,102.20,233385200
4,2010-07-06,20.00,16.11,6866900,103.64,102.87,256935300
...,...,...,...,...,...,...,...
2411,2020-01-28,568.49,566.90,11788500,325.06,326.89,63834000
2412,2020-01-29,575.69,580.99,17801500,328.38,326.62,53888900
2413,2020-01-30,632.42,640.81,29005700,324.36,327.68,75491800
2414,2020-01-31,640.00,650.57,15719300,327.00,321.73,113845600
